Milestone 5 - Model Training

Task 3: Train and tune other models

This task will focus on steps 3 to 5 of the workflow mentioned in the first task of this milestone. There are many different models that you can use to train your model. You can use KNN, decision trees, random forests... You have to tune them before making a decision, and not the other way around. So first, tune all of them, and then check which one performs better on the testing set. Remember not to overfit! Some models, like decision tree, are prone to overfitting, so even if they perform very well on the training set, make sure that it can also perform well on the testing set. If that's the case, you can add different ways of regularisation. Once you picked the model, save the model as model.joblib.

In [1]:
import re
import pandas as pd
import os
import numpy as np
from csv import reader
import plotly.express as px
import missingno as msno
import pickle

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import MinMaxScaler
from numpy import set_printoptions

from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

pd.options.mode.chained_assignment = None

In [2]:
# READ IN cleaned_dataset.csv
full_pd = pd.read_csv("cleaned_dataset_b.csv")
full_pd

,League,Season,Round,Home_Team,Away_Team,Elo_home,Elo_away,HOMETEAM_HOME_GOAL_SO_FAR,HOMETEAM_AWAY_GOAL_SO_FAR,AWAYTEAM_HOME_GOAL_SO_FAR,AWAYTEAM_AWAY_GOAL_SO_FAR,HOME_LASTEST_GOAL_DIFF,AWAY_LASTEST_GOAL_DIFF,Result
0,championship,2021,4,Coventry City,AFC Bournemouth,46.0,62.0,3,2,4,2,0,2,0
1,championship,2021,4,Norwich City,Derby County,62.0,60.0,2,2,0,6,0,-7,0
2,championship,2021,4,Blackburn Rovers,Cardiff City,58.0,60.0,5,0,1,4,8,-1,0
3,championship,2021,4,Luton Town,Wycombe Wanderers,51.0,41.0,2,1,0,3,1,-8,1
4,championship,2021,4,Middlesbrough,Barnsley,61.0,46.0,1,1,0,1,-1,-3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111647,serie_b,1997,38,Pescara,Padova,59.0,54.0,32,15,22,15,5,-3,0
111648,serie_b,1997,38,Genoa,Palermo FC,61.0,58.0,33,12,24,24,2,-1,1
111649,serie_b,1997,38,Torino,Ravenna FC,63.0,54.0,27,23,22,18,-2,-2,0
111650,serie_b,1997,38,Salernitana,Reggina,52.0,52.0,20,7,23,18,-2,3,0


In [3]:
# Create functions to filter different league
def getLeagueData(data, league, season=None):
    if season is None:
        league_pd =  data[(data["League"]==league)]
    else:
        league_pd =  data[(data["League"]==league) & (data["Season"]==season)]
    return league_pd

In [4]:
def get_ELO_diff(record):
    hscore = record['Elo_home']
    ascore = record['Elo_away']
    return (hscore - ascore)

In [5]:
def get_recent_goal_diff_diff(record):
    hscore = record['HOME_LASTEST_GOAL_DIFF']
    ascore = record['AWAY_LASTEST_GOAL_DIFF']
    return hscore - ascore

In [6]:
def get_home_away_total_goal_diff(record):
    hgoal = record['HOMETEAM_HOME_GOAL_SO_FAR']
    agoal = record['AWAYTEAM_AWAY_GOAL_SO_FAR']
    return hgoal - agoal

In [10]:
def tryModels(X, y):
    test_size = 0.3
    seed = 42
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

    # Model creation
    print("LogisticRegression")
    model = LogisticRegression() 
    model.fit(X_train, Y_train)
    result = model.score(X_train, Y_train) 
    print("Accuracy for train: %.3f%%" % (result*100.0))
    result = model.score(X_test, Y_test) 
    print("Accuracy for test: %.3f%%" % (result*100.0))
    print()

    # KNN 
    print("KNN")
    knn = KNeighborsClassifier(n_neighbors=15)
    knn.fit(X_train, Y_train)
    result = knn.score(X_train, Y_train) 
    print("Accuracy for train: %.3f%%" % (result*100.0))
    result = knn.score(X_test, Y_test) 
    print("Accuracy for test: %.3f%%" % (result*100.0))
    print()

    # decision trees  
    print("decision trees")
    clf = DecisionTreeClassifier()
    clf.fit(X_train, Y_train)
    result = clf.score(X_train, Y_train) 
    print("Accuracy for train: %.3f%%" % (result*100.0))
    result = clf.score(X_test, Y_test) 
    print("Accuracy for test: %.3f%%" % (result*100.0))
    print()

    # random forests
    print("random forests")
    model = RandomForestClassifier()
    model.fit(X_train, Y_train)
    result = model.score(X_train, Y_train) 
    print("Accuracy for train: %.3f%%" % (result*100.0))
    result = model.score(X_test, Y_test) 
    print("Accuracy for test: %.3f%%" % (result*100.0))
    print()

In [11]:
# load all directory as league name list
dir = "./Results"
leagues = [name for name in os.listdir(dir) if os.path.isdir(os.path.join(dir, name))]

# loop to open csv
result_with_goal_sofar_pd = pd.DataFrame()
for league in leagues:
    model_pd = getLeagueData(full_pd, league)
    model_pd = model_pd.dropna()

    if (model_pd.shape[0]==0):
        continue

    elo_diff_pd = model_pd.apply(get_ELO_diff, axis=1)
    model_pd.drop('Elo_home', inplace=True, axis=1)
    model_pd.drop('Elo_away', inplace=True, axis=1)
    model_pd.insert(loc=5, column="ELO_DIFF", value=elo_diff_pd.astype('Int64')) 
    
    recent_perf_diff_pd = model_pd.apply(get_recent_goal_diff_diff, axis=1)
    model_pd.drop('HOME_LASTEST_GOAL_DIFF', inplace=True, axis=1)
    model_pd.drop('AWAY_LASTEST_GOAL_DIFF', inplace=True, axis=1)
    model_pd.insert(loc=6, column="RECENT_PERF_DIFF", value=recent_perf_diff_pd.astype('Int64')) 

    goal_diff_pd = model_pd.apply(get_home_away_total_goal_diff, axis=1)
    model_pd.drop('HOMETEAM_HOME_GOAL_SO_FAR', inplace=True, axis=1)
    model_pd.drop('HOMETEAM_AWAY_GOAL_SO_FAR', inplace=True, axis=1)
    model_pd.drop('AWAYTEAM_HOME_GOAL_SO_FAR', inplace=True, axis=1)
    model_pd.drop('AWAYTEAM_AWAY_GOAL_SO_FAR', inplace=True, axis=1)
    model_pd.insert(loc=7, column="HOME_AWAY_GOAL_DIFF", value=recent_perf_diff_pd.astype('Int64')) 

    # delete no value column
    model_pd.drop('League', inplace=True, axis=1)
    model_pd.drop('Season', inplace=True, axis=1)
    model_pd.drop('Round', inplace=True, axis=1)
    model_pd.drop('Home_Team', inplace=True, axis=1)
    model_pd.drop('Away_Team', inplace=True, axis=1)

    array = model_pd.values
    X = array[:,0:(array.shape[1]-1)].astype('int')
    y = array[:,(array.shape[1]-1)].astype('int')

    # Scaler
    scaler = MinMaxScaler(feature_range=(0, 8))
    rescaledX = scaler.fit_transform(X)

    # summarize transformed data
    set_printoptions(precision=3)

    # Or Standardize
    #scaler = StandardScaler().fit(X)
    #rescaledX = scaler.transform(X)

    print()
    print()
    print(league)
    print("-------------------------------------")
    tryModels(rescaledX, y)
    



championship
-------------------------------------
LogisticRegression
Accuracy for train: 55.962%
Accuracy for test: 57.025%

KNN
Accuracy for train: 60.897%
Accuracy for test: 54.111%

decision trees
Accuracy for train: 71.186%
Accuracy for test: 54.036%

random forests
Accuracy for train: 71.186%
Accuracy for test: 54.858%



primeira_liga
-------------------------------------
LogisticRegression
Accuracy for train: 64.117%
Accuracy for test: 62.669%

KNN
Accuracy for train: 67.468%
Accuracy for test: 62.137%

decision trees
Accuracy for train: 70.398%
Accuracy for test: 60.213%

random forests
Accuracy for train: 70.398%
Accuracy for test: 60.008%



ligue_1
-------------------------------------
LogisticRegression
Accuracy for train: 60.556%
Accuracy for test: 58.078%

KNN
Accuracy for train: 63.293%
Accuracy for test: 57.530%

decision trees
Accuracy for train: 66.528%
Accuracy for test: 56.337%

random forests
Accuracy for train: 66.528%
Accuracy for test: 55.563%



segunda_divi

In [44]:
# try to add regularisation
model_pd = getLeagueData(full_pd, "serie_b", 2011)
model_pd

,League,Season,Round,Home_Team,Away_Team,Elo_home,Elo_away,HOMETEAM_HOME_GOAL_SO_FAR,HOMETEAM_AWAY_GOAL_SO_FAR,AWAYTEAM_HOME_GOAL_SO_FAR,AWAYTEAM_AWAY_GOAL_SO_FAR,HOME_LASTEST_GOAL_DIFF,AWAY_LASTEST_GOAL_DIFF,Result
103305,serie_b,2011,4,Padova,Reggina,53.0,58.0,1,1,2,0,-1,1,1
103306,serie_b,2011,4,Vicenza,Livorno,57.0,62.0,4,1,1,5,2,-1,0
103307,serie_b,2011,4,Siena,Cittadella,64.0,52.0,2,1,2,3,1,-3,1
103308,serie_b,2011,4,Sassuolo,Torino,54.0,59.0,1,1,2,3,3,-2,0
103309,serie_b,2011,4,Piacenza,Ascoli,59.0,60.0,1,1,3,1,-3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103729,serie_b,2011,42,Cittadella,Pescara,52.0,45.0,26,24,24,19,0,-2,1
103730,serie_b,2011,42,Varese,Piacenza,46.0,58.0,32,10,32,32,-2,-2,1
103731,serie_b,2011,42,Sassuolo,Reggina,54.0,58.0,24,20,21,11,-1,1,1
103732,serie_b,2011,42,AlbinoLeffe,Siena,55.0,64.0,31,30,45,14,-1,4,1


In [45]:
model_pd = model_pd.dropna()

elo_diff_pd = model_pd.apply(get_ELO_diff, axis=1)
model_pd.drop('Elo_home', inplace=True, axis=1)
model_pd.drop('Elo_away', inplace=True, axis=1)
model_pd.insert(loc=5, column="ELO_DIFF", value=elo_diff_pd.astype('Int64')) 

recent_perf_diff_pd = model_pd.apply(get_recent_goal_diff_diff, axis=1)
model_pd.drop('HOME_LASTEST_GOAL_DIFF', inplace=True, axis=1)
model_pd.drop('AWAY_LASTEST_GOAL_DIFF', inplace=True, axis=1)
model_pd.insert(loc=6, column="RECENT_PERF_DIFF", value=recent_perf_diff_pd.astype('Int64')) 

goal_diff_pd = model_pd.apply(get_home_away_total_goal_diff, axis=1)
model_pd.drop('HOMETEAM_HOME_GOAL_SO_FAR', inplace=True, axis=1)
model_pd.drop('HOMETEAM_AWAY_GOAL_SO_FAR', inplace=True, axis=1)
model_pd.drop('AWAYTEAM_HOME_GOAL_SO_FAR', inplace=True, axis=1)
model_pd.drop('AWAYTEAM_AWAY_GOAL_SO_FAR', inplace=True, axis=1)
model_pd.insert(loc=7, column="HOME_AWAY_GOAL_DIFF", value=recent_perf_diff_pd.astype('Int64')) 

# delete no value column
model_pd.drop('League', inplace=True, axis=1)
model_pd.drop('Season', inplace=True, axis=1)
model_pd.drop('Round', inplace=True, axis=1)
model_pd.drop('Home_Team', inplace=True, axis=1)
model_pd.drop('Away_Team', inplace=True, axis=1)

array = model_pd.values
X = array[:,0:(array.shape[1]-1)].astype('int')
y = array[:,(array.shape[1]-1)].astype('int')

# Scaler
scaler = MinMaxScaler(feature_range=(0, 8))
rescaledX = scaler.fit_transform(X)

# summarize transformed data
set_printoptions(precision=3)

test_size = 0.3
seed = 42
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

model = RandomForestClassifier()
model.fit(X_train, Y_train)
result = model.score(X_train, Y_train) 
print("Accuracy for train: %.3f%%" % (result*100.0))
result = model.score(X_test, Y_test) 
print("Accuracy for test: %.3f%%" % (result*100.0))
print()

Accuracy for train: 89.000%
Accuracy for test: 54.264%



In [46]:
from sklearn.tree import export_graphviz
import pydot

# Extract the small tree
tree_small = model.estimators_[3]
    
# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = ["ELO_DIFF","RECENT_PERF_DIFF","HOME_AWAY_GOAL_DIFF"], rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png');

In [49]:
model = RandomForestClassifier(n_estimators=10, max_depth = 4)
model.fit(X_train, Y_train)
result = model.score(X_train, Y_train) 
print("Accuracy for train: %.3f%%" % (result*100.0))
result = model.score(X_test, Y_test) 
print("Accuracy for test: %.3f%%" % (result*100.0))
print()

Accuracy for train: 65.667%
Accuracy for test: 52.713%



In [50]:
from sklearn.tree import export_graphviz
import pydot

# Extract the small tree
tree_small = model.estimators_[3]
    
# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree_re.dot', feature_names = ["ELO_DIFF","RECENT_PERF_DIFF","HOME_AWAY_GOAL_DIFF"], rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree_re.dot')
graph.write_png('small_tree_re.png');

In [51]:
# Save the model
from joblib import dump, load
dump(model, 'baseline_t3.joblib')

['baseline_t3.joblib']